In [ ]:
# Não esqueça de instalar o fitz, Pillow e o PyMuPDF

In [1]:
import tkinter as tk  # Biblioteca para criar interfaces gráficas
from tkinter import filedialog, messagebox  # Funções para dialog de arquivos e mensagens de erro
import fitz  # PyMuPDF, usada para manipulação de PDFs
import os  # Biblioteca para operações com arquivos e caminhos
from PIL import Image, ImageTk  # Biblioteca Pillow para lidar com imagens

# Função para selecionar o arquivo PDF
def selecionar_pdf():
    global pdf_path
    pdf_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])  # Janela para selecionar arquivo PDF
    if pdf_path:  # Verifica se um arquivo foi selecionado
        label_pdf.config(text=f"PDF selecionado: {os.path.basename(pdf_path)}")  # Atualiza label com o nome do PDF

# Função para selecionar o arquivo de cognatos
def selecionar_cognatos():
    global cognatos_path
    cognatos_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])  # Janela para selecionar arquivo de texto
    if cognatos_path:  # Verifica se um arquivo foi selecionado
        label_cognatos.config(text=f"Cognatos selecionados: {os.path.basename(cognatos_path)}")  # Atualiza label com o nome do arquivo

# Função principal que faz a análise do PDF
def analisar_pdf():
    # Verifica se ambos os arquivos foram selecionados
    if not pdf_path or not cognatos_path:
        messagebox.showerror("Erro", "Selecione o arquivo PDF e o arquivo de cognatos antes de continuar.")
        return
    
    try:
        # Ler os cognatos do arquivo de texto
        with open(cognatos_path, 'r', encoding='utf-8') as file:
            cognatos = file.read().splitlines()  # Cada linha do arquivo vira um item na lista de cognatos

        # Abrir o arquivo PDF
        doc = fitz.open(pdf_path)
        
        # Percorrer todas as páginas do PDF
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            
            # Iterar sobre os cognatos e buscar essas palavras na página
            for cognato in cognatos:
                areas = page.search_for(cognato)  # Procura a palavra na página
                for area in areas:
                    # Destaca a palavra encontrada com cor amarela
                    highlight = page.add_highlight_annot(area)
                    highlight.set_colors(stroke=(1, 1, 0))  # Amarelo
                    highlight.update()

        # Solicita ao usuário onde salvar o PDF modificado
        output_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
        if output_path:
            doc.save(output_path)  # Salva o arquivo com os destaques
            messagebox.showinfo("Sucesso", f"Arquivo salvo em: {output_path}")  # Mostra mensagem de sucesso
        doc.close()  # Fecha o documento

    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro: {e}")  # Mostra uma mensagem de erro em caso de exceção

# Criação da interface gráfica (Janela principal)
root = tk.Tk()
root.title("Analisador de Cognatos")  # Título da janela
root.geometry("500x350")  # Tamanho da janela ajustado para mais espaço
root.configure(bg="#f8f9fa")  # Cor de fundo suave (cinza claro)

# Carregar o ícone PNG e definir na janela
img = Image.open("duvidas.png")  # Abrir a imagem PNG
icon = ImageTk.PhotoImage(img)  # Converter para PhotoImage
root.tk.call('wm', 'iconphoto', root._w, icon)  # Definir como ícone da janela

# Variáveis globais para armazenar os caminhos dos arquivos selecionados
pdf_path = None
cognatos_path = None

# Título principal da aplicação
titulo = tk.Label(root, text="Analisador de Cognatos em PDF", font=("Arial", 18, "bold"), bg="#f8f9fa")
titulo.pack(pady=15)

# Labels e botões para selecionar o arquivo PDF
label_pdf = tk.Label(root, text="Selecione o arquivo PDF", font=("Arial", 12), bg="#f8f9fa")
label_pdf.pack(pady=5)

btn_pdf = tk.Button(root, text="Selecionar PDF", command=selecionar_pdf, width=25, font=("Arial", 12), bg="#007BFF", fg="white", relief="groove")
btn_pdf.pack(pady=10)

# Labels e botões para selecionar o arquivo cognatos.txt
label_cognatos = tk.Label(root, text="Selecione o arquivo cognatos.txt", font=("Arial", 12), bg="#f8f9fa")
label_cognatos.pack(pady=5)

btn_cognatos = tk.Button(root, text="Selecionar cognatos.txt", command=selecionar_cognatos, width=25, font=("Arial", 12), bg="#007BFF", fg="white", relief="groove")
btn_cognatos.pack(pady=10)

# Botão para iniciar a análise do PDF
btn_analisar = tk.Button(root, text="Analisar PDF", command=analisar_pdf, width=30, height=2, font=("Arial", 14), bg="#28A745", fg="white", relief="raised")
btn_analisar.pack(pady=25)

# Executa a interface
root.mainloop()
